In [1]:
import os
import networkx as nx
import osmnx as ox
import geopandas as gpd

In [2]:
ox_crs = 4326
local_crs = 32636
example_data_path = './data'

city_geometry = gpd.read_parquet(os.path.join(example_data_path, "city_geometry.parquet")).to_crs(ox_crs)

In [3]:
from masterplan_tools.preprocessing import GraphGenerator
intermodal_graph = GraphGenerator(city_geometry=city_geometry, local_crs=local_crs).get_graph()

In [4]:
nx.write_graphml(intermodal_graph, './data/graph.graphml')

In [5]:
blocks = gpd.read_parquet(os.path.join(example_data_path, "blocks_cutter_result.parquet")).to_crs(local_crs)

In [6]:
from shapely import Point

nodes_gdf = gpd.GeoDataFrame(intermodal_graph.nodes(data=True), columns=['node', 'data'])
nodes_gdf['geometry'] = nodes_gdf['data'].apply(lambda d : Point(d['x'], d['y']))
nodes_gdf.set_geometry('geometry',inplace=True)
nodes_gdf.set_crs(epsg=local_crs, inplace=True)
nodes_gdf.drop(labels=['data'], inplace=True, axis=1)

In [7]:
blocks['geometry'] = blocks['geometry'].apply(lambda x : x.representative_point())
blocks = blocks.loc[blocks['landuse']=='buildings']
blocks.explore()

In [8]:
import pandas as pd

sjoin = blocks.sjoin_nearest(nodes_gdf, how="left", distance_col="distance")
sjoin['weight'] = sjoin['distance']/(1000*4/60)
G = nx.MultiDiGraph(intermodal_graph)

def get_nodes(loc) -> dict[str, any]:
  nodes = []
  if isinstance(loc, pd.Series):
    nodes.append({
      'node': loc['node'],
      'weight': loc['weight']
    })
  if isinstance(loc, gpd.GeoDataFrame):
    tmp_loc = loc.copy().reset_index()
    for k in tmp_loc.index:
      nodes.append({
        'node': tmp_loc.loc[k,'node'],
        'weight': tmp_loc.loc[k,'weight']
      })
  return nodes

for i in sjoin.index:
  nodes = get_nodes(sjoin.loc[i])
  for node in nodes:
    G.add_edge(f'block_{i}', node['node'], node['weight'])
    G.add_edge(node['node'], f'block_{i}', node['weight'])

In [9]:
from shapely import LineString

def path_to_linestring(graph, path):
  points = []
  for node in path:
    data = graph.nodes[node]
    points.append(Point(data['x'], data['y']))
  return LineString(points)

linestrings = []
block_id = 115
for i in blocks.index:
  if i==block_id : continue
  try:
      linestrings.append(path_to_linestring(G, nx.shortest_path(G, f"block_{block_id}", f"block_{i}")[1:-1]))
  except: ...
gdf = gpd.GeoDataFrame({'geometry':linestrings})

In [10]:
gdf.set_crs(epsg=local_crs).explore()